In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import idaes

import numpy as np
from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations

from fim_doe import *
from pyomo.contrib.sensitivity_toolbox.sens import sipopt

## Define fixed bed model 

In [2]:
from fixed_bed_model_indexed_dynamic import *

alpha option:
1 / (1 + exp(-x))
    Import ComponentSet from pyomo.common.collections.  (deprecated in 5.7.1)
    (called from <frozen importlib._bootstrap>:228)


In [3]:
def add_model(doe, timepoints, time_start=0):
    '''
    add variables, equations and discretize the model time
    Argument:
        timepoints: the timesteps
        time_start: where the timesteps start. For self defined timepoints it's 0. For experimental data it's -10.
    '''
    add_variables(doe, timesteps=timepoints, start=time_start)
    
    add_equations(doe)
    
    print ('the number of timepoints is', NFEt)
    
def discretize(doe, no_points=68, collo=False):
    '''
    no_points: how many time invertals to divide. For self defined timepoints it's 69. For experimental data it's 68. 
    here it's 68
    '''
    if collo:
        TransformationFactory('dae.collocation').apply_to(doe, nfe=no_points, ncp=3, scheme='LAGRANGE-RADAU', wrt=doe.t)
    else:
        TransformationFactory('dae.finite_difference').apply_to(doe, nfe=no_points, scheme='BACKWARD', wrt=doe.t)
    
    
def initialize(doe, init_point):
    '''
    Argument: 
        init_point: initial point, csv file
    '''
    # Initialize 
    print('The init point is', init_point)    
    store = pd.read_csv(init_point)
    position_max = store['position'].max()
    store['position'] = store['position'] / position_max

    initial_bed_csv(doe, store)
    fix_initial_bed(doe)
    
def generate_exp(feed, bath, y, const_q):
    initialize_q = {}
    for t in exp_time_:
        initialize_q[t] = const_q
    print(exp_time_[0])
    dv_dict_overall = {'temp_feed': {0: feed}, 'temp_bath': {0:bath}, 'yfeed': {0: y}, 'Q': initialize_q}
    return dv_dict_overall

In [4]:
NFEt = 68
time_scale = 3200
collo = False
exp_time_ = time_points(NFEt, time_scale)

scena_all = {'scena-name': [0], 'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

#initialize_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/20200729_energy.csv'
#initialize_point = '2021oct8_f313_b313_y15_5e6.csv'
initialize_point = '2021oct8_f293_b293_y15_5e6.csv'
#initialize_point = './addQfigures_modeldata/Q90k_293_293_15_5e30.csv'
#initialize_point = 'Q90k_293_293_15_5e30.csv'
#initialize_point = '2021oct8_f303_b293_y40_5e3.csv'
#initialize_point = './Q0k_313_293_15_5e31.csv'
#initialize_point = './priors/20210916_feed293_bath293_5e3.csv'
#init_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/fixed_bed_initial/T353y20_5e3.csv'
#initialize_point = './priors/20210916_feed313_bath293_5e3.csv'
#init_point = 'init_test.csv'
#init_point = 'T353y20_5e3.csv'

In [5]:
def model_integrate(scena, args_list, exp_time=exp_time_, init_point=initialize_point):
    test = create_model(scena, temp_feed=293.15, temp_bath=293.15, y=0.15, doe_model=args_list[0], k_aug=args_list[1], opt=False, diff=0)
    add_model(test, exp_time)
    discretize(test)
    initialize(test, init_point)
    return test


In [6]:
# choose from model and k_aug 
model_opt = 'doe'

if (model_opt == 'doe'):
    args_ = [True, False]
    
elif (model_opt =='kaug'):
    args_ = [True, True]
    

parameter_dict = {'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

#dv_pass = {'temp_feed': [0], 'temp_bath':[0], 'yfeed': [0]}
#dv_pass = {'temp_feed': None, 'temp_bath':None , 'yfeed': None}
dv_pass = {'temp_feed': None, 'yfeed': None, 'Q':exp_time_}

#measurement = ['FCO2', 'temp']
#measurement_extra = {'FCO2': [19], 'temp': [10, 19]}

t_measure = exp_time_
#measurement_pass = {'FCO2': t_measure, 'temp': t_measure}
measurement_pass = {'FCO2': {19: t_measure}, 'temp': {10: t_measure, 19:t_measure}}

measure_class = Measurements(measurement_pass)

In [7]:
# empty prior
prior_all = np.zeros((2,2))

prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[0. 0.]
 [0. 0.]]
Prior Det: 0.0
Eigenvalue of the prior experiments FIM: [0. 0.]
Eigenvalue of the prior experiments FIM: [[1. 0.]
 [0. 1.]]


In [8]:
fim_init = [[0.0029331 , 0.01236752],
 [0.01236752 , 0.71783208]]

#f2 = open('jac_initial', 'rb')
#jac_init = pickle.load(f2)
#f2.close()

#jac_init = jac_init_1.copy()
    
l_init = np.linalg.cholesky(fim_init)
print(l_init)

[[0.0541581  0.        ]
 [0.22835955 0.8158946 ]]


In [ ]:
import os

scores = {} # scores is an empty dict already
target='jac_initial'
if os.path.getsize(target) > 0:      
    with open(target, "rb") as f:
        unpickler = pickle.Unpickler(f)
        # if file is not empty scores will be equal
        # to the value unpickled
        scores = unpickler.load()
        
print(scores)

## Compute FIM

In [9]:
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
#sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

# feed, bath, y
exp1 = generate_exp(293.15, 293.15, 0.15, 0)

#dv_pass = {'temp_feed': None, 'yfeed': None, 'temp_bath': None, 'Q':exp_time_}

0


In [ ]:
#q_list = [0, 10, 20, 30, 40, 50, 60,70,80,90,100]

q_list = [120,130,140,150] 

for i in q_list: 
    exp1 = generate_exp(293.15, 293.15, 0.15, i*1000)
    output_name = 'Q'+str(i)+'k_293_293_15_5e3'
    
    doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement_pass, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_)

    if_s = True 

    result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                                   scale_nominal_param_value = if_s, 
                                    store_output = output_name, read_output=None,
                                    extract_single_model = extract3_v2,
                                     formula='central')


In [10]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_)

if_s = True 

result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                               scale_nominal_param_value = if_s, 
                                store_output = 'store_output', read_output=None,
                                extract_single_model = extract3_v2,
                                formula='central')

if sensi_opt == 'simultaneous_finite':
    result.extract_FIM(doe_object.m, doe_object.design_timeset,
                                                          doe_object.square_result, doe_object.obj_opt)
else: 
    result.calculate_FIM(doe_object.jac, doe_object.design_values)
    
    

All measurements are flattened.
Flatten measurement name: ['FCO2_index_19', 'temp_index_10', 'temp_index_19']
Flatten measurement timeset: {'FCO2_index_19': [0, 47.05882352941177, 94.11764705882354, 141.1764705882353, 188.23529411764707, 235.29411764705884, 282.3529411764706, 329.4117647058824, 376.47058823529414, 423.5294117647059, 470.5882352941177, 517.6470588235295, 564.7058823529412, 611.7647058823529, 658.8235294117646, 705.8823529411764, 752.9411764705881, 799.9999999999998, 847.0588235294115, 894.1176470588232, 941.1764705882349, 988.2352941176466, 1035.2941176470583, 1082.35294117647, 1129.4117647058818, 1176.4705882352935, 1223.5294117647052, 1270.5882352941169, 1317.6470588235286, 1364.7058823529403, 1411.764705882352, 1458.8235294117637, 1505.8823529411754, 1552.9411764705872, 1599.9999999999989, 1647.0588235294106, 1694.1176470588223, 1741.176470588234, 1788.2352941176457, 1835.2941176470574, 1882.3529411764691, 1929.4117647058808, 1976.4705882352926, 2023.5294117647043, 2

ApplicationError: Solver (ipopt) did not exit normally

In [ ]:
#print(doe_object.jac)
print((doe_object.jac_3D))
print(np.shape(doe_object.jac_3D))

In [ ]:
result.calculate_FIM(doe_object.jac, doe_object.design_values, jaco_involved=None)




In [ ]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

In [ ]:
dataframe = extract3_v2(doe_object.models[0])
dataframe.to_csv('test6.csv')

In [ ]:
print(doe_object.dsdp)

In [ ]:
import pickle

with open('store_output_botht', 'rb') as f:
    data = pickle.load(f)
    f.close()
print(data)

In [ ]:
doe_object.models[0]

In [ ]:
breakthrough_modify2(doe_object.models[0])

In [ ]:
print(value(doe_object.models[0].temp_feed))
print(value(doe_object.models[0].temp_bath))
print(value(doe_object.models[0].yfeed))
print(value(doe_object.models[0].Q[0]))

In [ ]:
def breakthrough_modify2(m, file=None, source="lab"):
    '''
    Draw breakthrough curve of the Pyomo model
    
    Arguments:
        m: Pyomo model
        file: when source = 'computer', this is where the computer experimental data is stored. Otherwise, its default is None
        source: if computer, it is plotting the computer experiment data. If lab, its comparing the experiments from the lab
    
    Return: None 
    
    Other: plot
    
    '''
    
    
    # unit: mol/m3
    outlet_den, _, _, _ = extract3d(m, m.C, 'CO2')
    
    # unit: cm/s
    outlet_vel, _, _, _ = extract2d(m, m.v)
    
    outlet_n2, _, _, _ = extract3d(m, m.C, 'N2')
    
    model_temp, _,_,_ = extract2d(m,m.temp)
    
    
    print(np.shape(model_temp))
    
    
    
    T = []
    for j in m.t:
        T.append(value(j))
        t_final = T[-1]
        
    model_q = []
    for i in T:
        model_q.append(value(m.Q[i]))
    print(model_q)
    
    if source == "lab":
        break_wvu = pd.read_csv('/Users/wangjialu/dowlinglab/sorption-fixed-bed/breakthrough_wvu.csv')
    
        exp = pd.read_csv('/Users/wangjialu/dowlinglab/sorption-fixed-bed/co2_breakthrough.csv')
        
        data_c1 = exp['FCO2']
        data_c = np.asarray(data_c1)
        data_t1 = exp['time']  
        data_t = np.asarray(data_t1*60+10)
        new_data = np.interp(T, data_t, data_c)
    
        fco2 = []
        for t in m.t:
            fco2.append(value(m.FCO2[0,19,t]))
        
        yco2 = []
        for i in range(len(fco2)):
            yco2.append(fco2[i]/fco2[-1])
            
        # real value of FCO2. for verification
        comp = outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1])
    
        plt.plot(exp['time']*60, exp['FCO2'],'b.', color='r', label='Experimental data')
        #plt.plot(T, outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1]), label='Model prediction')
        plt.plot(T, yco2, label='Model prediction')
        #plt.plot(T, yco2_real, label='Real prediction')
        plt.plot()
        
        plt.xlabel('time [s]')
        plt.ylabel('Normalized outlet gas density of CO\N{SUBSCRIPT TWO} ')
        plt.title('Breakthrough curve')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()
        
        
        plt.plot(T, model_temp[10], label='Middle bed T')
        plt.plot(T, model_temp[19], label='Outlet end T')
        plt.xlabel('time [s]')
        plt.ylabel('Temperature [K]')
        plt.legend()
        plt.show()
        
        plt.plot(T, model_q)
        plt.xlabel('time [s]')
        plt.ylabel('Q [J/m$^3$/s]')
        plt.show()
        
        
    elif source == "computer": 
        
        sol = pd.read_csv(file)
    
        #for i in range(len(time_exp)):    
        #    yco2_exp[time_exp[i]] = sol['FCO2'][i]
        #    temp_mid_exp[time_exp[i]] = sol['temp_mid'][i]
        #    temp_end_exp[time_exp[i]] = sol['temp_end'][i]
        
        exp_fco2 = np.asarray(sol['FCO2'].values.tolist())
        exp_temp_mid = np.asarray(sol['temp_mid'].values.tolist())
        exp_temp_end = np.asarray(sol['temp_end'].values.tolist())
        
        plt.plot(T, exp_fco2/exp_fco2[-1], label = 'Experimental data')
        plt.plot(T, outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1]), label='Model prediction')
        plt.xlabel('time [s]')
        plt.ylabel('Normalized outlet gas density of CO\N{SUBSCRIPT TWO} ')
        plt.title('Breakthrough curve')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()
        
        
        plt.plot(T, exp_temp_mid, label='Experimental data of middle T')
        plt.plot(T, model_temp[10,:], label='Model prediction of middle T')
        plt.plot(T, exp_temp_end, label='Experimental data of end T')
        plt.plot(T, model_temp[19,:], label='Model prediction of end T')
        plt.xlabel('time [s]')
        plt.ylabel('Temperature [K]')
        plt.title('Temperature model prediction and experimental data')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()

## Gradient-based optimization 

In [ ]:
# feed, bath, y
exp1 = generate_exp(303.15, 293.15, 0.4, 10000)


### Sequential initial solution 

In [ ]:
sensi_opt = 'sequential_finite'

doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement_pass, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_)
if_s = True 

result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                               scale_nominal_param_value = if_s, 
                                store_output = 'store_output', read_output=None,
                                formula='central')


In [ ]:
from pyomo.core.kernel.component_set import ComponentSet

def large_residuals_set(block, tol=1e-5, LOUD=True):
    """
    Method to return a ComponentSet of all Constraint components with a
    residual greater than a given threshold which appear in a model.
    Args:
        block : model to be studied
        tol : residual threshold for inclusion in ComponentSet
    Returns:
        A ComponentSet including all Constraint components with a residual
        greater than tol which appear in block
    """
    large_residuals_set = ComponentSet()
    for c in block.component_data_objects(
            ctype=Constraint, active=True, descend_into=True):
        if c.active and value(c.lower - c.body()) > tol:
            large_residuals_set.add(c)
        elif c.active and value(c.body() - c.upper) > tol:
            large_residuals_set.add(c)
    
    if LOUD:
        print("All equality constraints with residuals larger than",tol)
        for c in large_residuals_set:
            print(c)
            
    return large_residuals_set

large_residuals_set(doe_object.models[0], tol=10)

In [ ]:
result.calculate_FIM(doe_object.jac, doe_object.jac_3D, doe_object.design_values, 
                     jaco_involved_name=['FCO2','temp'], jaco_involved_extra_index=[[19],[10,19]])




In [ ]:
print(np.log10(1194655292.8955846))

In [ ]:
fim_init = result.FIM
print(fim_init)

jac_init = doe_object.jac_3D.copy()
#print(jac_init)
    
l_init = np.linalg.cholesky(fim_init)
print(l_init)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement_pass, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_)

square_result, optimize_result = doe_object.optimize_doe(exp1, if_optimize=True, if_Cholesky=True, 
                                                         jac_involved_measurement = {'FCO2':[19], 'temp':[10,19]},
                                                         #jac_involved_measurement = {'FCO2': [10]}, 
                                                         scale_nominal_param_value=True, objective_option='det')
                                                         #L_initial=l_init, jac_initial=jac_init, fim_initial=fim_init)


In [ ]:
maomao =[[ 0.04242279 , 0.03639749],
 [ 0.03639749 , 14.6350451 ]]
print(np.log10(np.linalg.det(maomao)))

In [ ]:
large_residuals_set(doe_object.models[0], tol=10)

In [ ]:
jac_init_1 = doe_object.analysis_square.jac_extracted.copy()
print(type(jac_init_1))
print(jac_init_1[0,0,0])

print(jac_init_1)

fo = open('jac_initial', 'wb')
pickle.dump(jac_init_1, fo)
fo.close



In [ ]:
print('======Result summary======')
print('This optimization is solved with status:', optimize_result.status)
print('It gives solution:', optimize_result.solution)
print('The log10(OF) optimized is:', optimize_result.obj_value)
print('The result FIM is:', optimize_result.FIM)

t_list = []
for t in optimize_result.model.t:
    t_list.append(t)

T_list = []
for i in t_list:
    T_list.append(value(optimize_result.model.T[i]))

si=16
plt.rc('axes', titlesize=si)
plt.rc('axes', labelsize=si)
plt.rc('xtick', labelsize=si)
plt.rc('ytick', labelsize=si)
plt.rc('legend', fontsize=12)
    
plt.plot(t_list, T_list, 'b', linewidth=2)
#plt.scatter(t_list, T_list, 'b')
plt.ylabel('T [$K$]')
plt.xlabel('Time [$h$]')
plt.show()

## Heatmap

In [ ]:
design_ranges = [list(np.linspace(293,353,13)), list(np.linspace(0.1,0.4,13))]
print(design_ranges)

In [ ]:
#design_ranges = [list(np.linspace(293,333,5)), list(np.linspace(0.1,0.3,6))]
#design_ranges = [[293, 303, 313, 323, 333, 343], [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]]
#design_ranges = [[298, 308, 318, 328, 338, 348,353], [0.125, 0.175, 0.225, 0.275, 0.325, 0.375]]
#design_ranges = [[298, 308, 318, 328, 338, 348,353],[0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]]
#design_ranges = [[293, 303, 313, 323, 333, 343],[0.125, 0.175, 0.225, 0.275, 0.325, 0.375]]

dv_apply_name = ['temp_feed','yfeed']
#dv_apply_name = ['temp_feed', 'Q']

dv_apply_time = [[0],[0]]
#dv_apply_time = [[0], t_measure]

# Define experiments
exp1 = generate_exp(313.15, 293.15, 0.15, 0)
    
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
#sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

print(design_ranges)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement_pass, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None,  args=args_)

all_fim = doe_object.run_grid_search(exp1, design_ranges,dv_apply_name, dv_apply_time, 
                                     mode=sensi_opt, tee_option=True,
                                    scale_nominal_param_value=True, 
                                     store_name='20211031_heatmap'
                                    )



In [ ]:
doe_object1 = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement_pass, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None,  args=args_)
result = doe_object1.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                                   scale_nominal_param_value = True, 
                                    store_output = 'dont', read_output='20211028_heatmap0',
                                     formula='central')

In [ ]:
result.calculate_FIM(doe_object1.jac, doe_object1.design_values, jaco_involved=None)

print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

In [ ]:
print(np.log10(0.0011662257647824846))

In [ ]:
test = all_fim.extract_criteria()

fixed = {}
all_fim.figure_drawing(fixed, ['temp_feed','yfeed'], 'Fixed bed','$feed T [K]$', 'y' )

In [ ]:
na = ['temp_feed', 'yfeed']
dd = ['temp_feed', 'yfeed']
a = [[293.0, 303.0, 313.0, 323.0, 333.0], [0.1, 0.14, 0.18, 0.22, 0.26, 0.3]]

In [ ]:
print(a[na.index('temp_feed')])

In [ ]:
def extract3_v2(m):
    ''' 
    Extract results from Pyomo model for brute force DoE problem. 
    Compared to extract2: less options kept, add the design variable values 
    
    Arguments:
        m: the model
    
    Return: a single pandas dataframe storing all results
    '''
    nTime = len(m.t)
    nGrid = len(m.zgrid)
    n = nTime*nGrid
    
    ### Variables to be extracted
    
    # Two 3D variables, for COMPS(N2, CO2, He)
    
    FCO2_1, FCO2_2, FCO2_3, FCO2_4 = extract2d(m, m.FCO2)
    
    FCO21 = np.reshape(FCO2_1, n)
    FCO22 = np.reshape(FCO2_2, n)
    FCO23 = np.reshape(FCO2_3, n)
    FCO24 = np.reshape(FCO2_4, n)
    
    C_N2_1, C_N2_2, C_N2_3, C_N2_4 = extract3d(m,m.C,'N2')
    C_CO2_1, C_CO2_2, C_CO2_3, C_CO2_4 = extract3d(m,m.C,'CO2')
    
    C_N21 = np.reshape(C_N2_1, n)
    C_CO21 = np.reshape(C_CO2_1, n)
    
    C_N22 = np.reshape(C_N2_2, n)
    C_CO22 = np.reshape(C_CO2_2, n)
    
    C_N23 = np.reshape(C_N2_3, n)
    C_CO23 = np.reshape(C_CO2_3, n)
    
    C_N24 = np.reshape(C_N2_4, n)
    C_CO24 = np.reshape(C_CO2_4, n)
    
    
    dcdt_N2_1, dcdt_N2_2, dcdt_N2_3, dcdt_N2_4 = extract3d(m,m.dCdt,'N2')
    dcdt_CO2_1, dcdt_CO2_2, dcdt_CO2_3, dcdt_CO2_4 = extract3d(m,m.dCdt,'CO2')
    
    dcdt_N21 = np.reshape(dcdt_N2_1,n)
    dcdt_CO21 = np.reshape(dcdt_CO2_1,n)
    
    dcdt_N22 = np.reshape(dcdt_N2_2,n)
    dcdt_CO22 = np.reshape(dcdt_CO2_2,n)
    
    dcdt_N23 = np.reshape(dcdt_N2_3,n)
    dcdt_CO23 = np.reshape(dcdt_CO2_3,n)
    
    dcdt_N24 = np.reshape(dcdt_N2_4,n)
    dcdt_CO24 = np.reshape(dcdt_CO2_4,n)
    
    # Three 2D variables
    vel_1, vel_2, vel_3, vel_4 = extract2d(m, m.v)
    
    vel1 = np.reshape(vel_1, n)
    vel2 = np.reshape(vel_2,n)
    vel3 = np.reshape(vel_3,n)
    vel4 = np.reshape(vel_4,n)
    
    P_1, P_2, P_3, P_4 = extract2d(m,m.P)
    P1 = np.reshape(P_1,n)
    P2 = np.reshape(P_2, n)
    P3 = np.reshape(P_3, n)
    P4 = np.reshape(P_4, n)
    
    total_den_1, total_den_2, total_den_3, total_den_4 = extract2d(m,m.total_den)
    total_den1 = np.reshape(total_den_1, n)
    total_den2 = np.reshape(total_den_2, n)
    total_den3 = np.reshape(total_den_3, n)
    total_den4 = np.reshape(total_den_4, n)
    
    if m.energy:
        T_1, T_2, T_3, T_4 = extract2d(m,m.temp)
        T1 = np.reshape(T_1, n)
        T2 = np.reshape(T_2, n)
        T3 = np.reshape(T_3, n)
        T4 = np.reshape(T_4, n)
        
        dTdt_1, dTdt_2, dTdt_3, dTdt_4 = extract2d(m,m.dTdt)
        dTdt1 = np.reshape(dTdt_1,n)
        dTdt2 = np.reshape(dTdt_2,n)
        dTdt3 = np.reshape(dTdt_3,n)
        dTdt4 = np.reshape(dTdt_4,n)
        
        nplin_1, nplin_2, nplin_3, nplin_4 = extract2d(m, m.nplin)
        nplin1 = np.reshape(nplin_1, n)
        nplin2 = np.reshape(nplin_2, n)
        nplin3 = np.reshape(nplin_3, n)
        nplin4 = np.reshape(nplin_4, n)
        
    
    # Seven 3D variables, for SCOMPS(CO2)
    spp_1, spp_2, spp_3, spp_4 = extract3d(m, m.spp, 'CO2') 

    spp1 = np.reshape(spp_1,n)
    spp2 = np.reshape(spp_2,n)
    spp3 = np.reshape(spp_3,n)
    spp4 = np.reshape(spp_4,n)

    nchemstar_1, nchemstar_2, nchemstar_3, nchemstar_4 = extract3d(m, m.nchemstar,'CO2')

    nchemstar1 = np.reshape(nchemstar_1, n)
    nchemstar2 = np.reshape(nchemstar_2, n)
    nchemstar3 = np.reshape(nchemstar_3, n)
    nchemstar4 = np.reshape(nchemstar_4, n)
    

    dnchemdt_1, dnchemdt_2, dnchemdt_3, dnchemdt_4 = extract3d(m, m.dnchemdt,'CO2')
    dnchemdt1 = np.reshape(dnchemdt_1, n)
    dnchemdt2 = np.reshape(dnchemdt_2, n)
    dnchemdt3 = np.reshape(dnchemdt_3, n)
    dnchemdt4 = np.reshape(dnchemdt_4, n)
    

    nchem_1, nchem_2, nchem_3, nchem_4 = extract3d(m, m.nchem,'CO2')
    nchem1 = np.reshape(nchem_1, n)
    nchem2 = np.reshape(nchem_2, n)
    nchem3 = np.reshape(nchem_3, n)
    nchem4 = np.reshape(nchem_4, n)


    nphysstar_1, nphysstar_2, nphysstar_3, nphysstar_4 = extract3d(m, m.nphysstar,'CO2')
    nphysstar1 = np.reshape(nphysstar_1, n)
    nphysstar2 = np.reshape(nphysstar_2, n)
    nphysstar3 = np.reshape(nphysstar_3, n)
    nphysstar4 = np.reshape(nphysstar_4, n)

    dnphysdt_1, dnphysdt_2, dnphysdt_3, dnphysdt_4 = extract3d(m, m.dnphysdt,'CO2')
    dnphysdt1 = np.reshape(dnphysdt_1, n)
    dnphysdt2 = np.reshape(dnphysdt_2, n)
    dnphysdt3 = np.reshape(dnphysdt_3, n)
    dnphysdt4 = np.reshape(dnphysdt_4, n)

    nphys_1, nphys_2, nphys_3, nphys_4 = extract3d(m, m.nphys,'CO2')
    nphys1 = np.reshape(nphys_1, n)
    nphys2 = np.reshape(nphys_2, n)
    nphys3 = np.reshape(nphys_3, n)
    nphys4 = np.reshape(nphys_4, n)
    

    

    
    # Meshgrid
    x = []
    for t in m.t:
        x.append(value(t))
    
    y = []
    for z in m.zgrid:
        y.append(value(z))

    
    [X,Y] = np.meshgrid(x,y)
    time = np.reshape(X,n)
    space = np.reshape(Y,n)
    
    model_q = []
    for i in x:
        model_q.append(value(m.Q[i]))
    print(model_q)
   
    if m.chemsorb and m.physsorb:
        if m.energy:
            store = pd.DataFrame({'time': time,
                              'position':space,
                              #'T_inlet': m.temp_feed,
                              #'y_inlet': m.yfeed,
                              #'status': 0,
                              #'fco2': FCO21, 
                              'den_N2': C_N21,
                              'den_CO2': C_CO21,
                              'dcdt_N2':dcdt_N21,
                              'dcdt_CO2':dcdt_CO21,
                              'vel': vel1,
                              'pressure':P1,
                              'temp': T1,
                              'dTdt': dTdt1, 
                              'nplin': nplin1, 
                              'total_den':total_den1,
                              'solid_pres':spp1,
                              'nchem_eq': nchemstar1,
                              'nphys_eq': nphysstar1,
                              'dndt_chem': dnchemdt1,
                              'dndt_phys': dnphysdt1,
                              'nchem': nchem1,
                              'nphys':nphys1,
                              'fco2_k': FCO22, 
                              'den_N2_k': C_N22,
                              'den_CO2_k': C_CO22,
                              'dcdt_N2_k':dcdt_N22,
                              'dcdt_CO2_k':dcdt_CO22,
                              'vel_k': vel2,
                              'pressure_k':P2,
                              'temp_k': T2,
                              'total_den_k':total_den2,
                              'solid_pres_k':spp2,
                              'nchem_eq_k': nchemstar2,
                              'nphys_eq_k': nphysstar2,
                              'dndt_chem_k': dnchemdt2,
                              'dndt_phys_k': dnphysdt2,
                              'nchem_k': nchem2,
                              'nphys_k': nphys2,
                              'fco2_u': FCO23, 
                              'den_N2_u': C_N23,
                              'den_CO2_u': C_CO23,
                              'dcdt_N2_u':dcdt_N23,
                              'dcdt_CO2_u':dcdt_CO23,
                              'vel_u': vel3,
                              'pressure_u':P3,
                              'temp_u': T3,
                              'total_den_u':total_den3,
                              'solid_pres_u':spp3,
                              'nchem_eq_u': nchemstar3,
                              'nphys_eq_u': nphysstar3,
                              'dndt_chem_u': dnchemdt3,
                              'dndt_phys_u': dnphysdt3,
                              'nchem_u': nchem3,
                              'nphys_u': nphys3,
                              'fco2_f': FCO24, 
                              'den_N2_f': C_N24,
                              'den_CO2_f': C_CO24,
                              'dcdt_N2_f':dcdt_N24,
                              'dcdt_CO2_f':dcdt_CO24,
                              'vel_f': vel4,
                              'pressure_f':P4,
                              'temp_f': T4,
                              'total_den_f':total_den4,
                              'solid_pres_f':spp4,
                              'nchem_eq_f': nchemstar4,
                              'nphys_eq_f': nphysstar4,
                              'dndt_chem_f': dnchemdt4,
                              'dndt_phys_f': dnphysdt4,
                              'nchem_f': nchem4,
                              'nphys_f': nphys4
                             })
            
        else:
        # save the arrays into the data frame
        # TODO: add the third set 
            store = pd.DataFrame({'time': time,
                              'position':space,
                              'T_inlet': m.temp_feed,
                              'y_inlet': m.yfeed['CO2'],
                              'fco2': FCO21, 
                              'den_N2': C_N21,
                              'den_CO2': C_CO21,
                              'dcdt_N2':dcdt_N21,
                              'dcdt_CO2':dcdt_CO21,
                              'vel': vel1,
                              'pressure':P1,
                              'total_den':total_den1,
                              'solid_pres':spp1,
                              'nchem_eq': nchemstar1,
                              'nphys_eq': nphysstar1,
                              'dndt_chem': dnchemdt1,
                              'dndt_phys': dnphysdt1,
                              'nchem': nchem1,
                              'nphys':nphys1,
                              'den_N2_u': C_N22,
                              'den_CO2_u': C_CO22,
                              'dcdt_N2_u':dcdt_N22,
                              'dcdt_CO2_u':dcdt_CO22,
                              'vel_u': vel2,
                              'pressure_u':P2,
                              'total_den_u':total_den2,
                              'solid_pres_u':spp2,
                              'nchem_eq_u': nchemstar2,
                              'nphys_eq_u': nphysstar2,
                              'dndt_chem_u': dnchemdt2,
                              'dndt_phys_u': dnphysdt2,
                              'nchem_u': nchem2,
                              'nphys_u': nphys2
                             })
    
    else:
        print('check the adsorption options!!! not a square problem')
    
    return store